In [1]:
import numpy as np
import zarr, healpy as hp
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter1d
from pathlib import Path

In [ ]:
root = "/media/nick/cluster/projects/data/killi_tracker/"
project_name = "20250716"
projection_path = Path(root) / "output_data" / "sphere_projections" / project_name
well_list = sorted(projection_path.glob("*.zarr"))
well_ind = 0
well_zarr_path = well_list[well_ind]
values_key="raw"
t_index=32
channel=0
nside=None,

# --- load ---
g = zarr.open_group(str(well_zarr_path), mode="r")
arr = g[values_key]   # shape (T, C, Npix) or (T, Npix)
nval = arr.shape[-1]
if nside is None:
    nside = int(np.sqrt(nval/12))


if arr.ndim == 3:
    vals = np.array(arr[t_index, channel])
elif arr.ndim == 2:
    vals = np.array(arr[t_index])

nval = vals.shape[-1]
nside = int(np.sqrt(nval/12))
npix = hp.nside2npix(nside)

In [ ]:
# --- optional smoothing on HEALPix sphere ---
# vals = gaussian_filter1d(vals, sigma=3, mode="wrap")

# --- get pixel centers on sphere ---
theta, phi = hp.pix2ang(nside, np.arange(npix))
x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

# --- Plotly 3D scatter ---
fig = go.Figure(
    data=go.Scatter3d(
        x=x, y=y, z=z,
        mode="markers",
        marker=dict(
            size=2,
            color=vals,
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title=values_key)
        )
    )
)
fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode="data"
    ),
    title=f"{Path(well_zarr_path).name}  t={t_index}  ch={channel}"
)
fig.show()

In [ ]:
import numpy as np
import healpy as hp
import plotly.graph_objects as go

def healpix_surface_plot(values, nside, n_theta=200, n_phi=400, cmap="Viridis"):
    # Regular grid in (theta, phi)
    theta = np.linspace(0, np.pi, n_theta)
    phi   = np.linspace(0, 2*np.pi, n_phi)
    TH, PH = np.meshgrid(theta, phi, indexing="ij")

    # Interpolate HEALPix values onto the grid
    vals_grid = hp.get_interp_val(values, TH.ravel(), PH.ravel(), nest=False)
    vals_grid = vals_grid.reshape(TH.shape)

    # Cartesian coords for unit sphere
    X = np.sin(TH) * np.cos(PH)
    Y = np.sin(TH) * np.sin(PH)
    Z = np.cos(TH)

    fig = go.Figure(
        data=go.Surface(
            x=X, y=Y, z=Z,
            surfacecolor=vals_grid,
            colorscale=cmap,
            cmin=np.nanmin(vals_grid),
            cmax=np.nanmax(vals_grid),
            showscale=True
        )
    )
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            aspectmode="data"
        )
    )
    return fig


In [ ]:
fig = healpix_surface_plot(vals, nside, n_theta=100, n_phi=200)
fig.show()